In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Activation, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, BatchNormalization
from keras.optimizers import SGD
from keras.applications.vgg16 import VGG16
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from keras.regularizers import l2
from keras.optimizers import Adam

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                            patience=4,
                                            factor=0.5,
                                            min_lr = 0.001,
                                            verbose = 1)

early_stoping = EarlyStopping(monitor='val_loss',patience= 3,restore_best_weights=True,verbose=0)

input_shape=(320, 320, 1)
reshaped_shape = (1, 320, 320, 1)
l2_rate = 0.01


def schedule(epoch, lr):
    if epoch <= 10:
        return 0.001
    else:
        return 0.0001

lr_schedule = LearningRateScheduler(schedule)
adam = Adam(learning_rate=0.01)


model = Sequential()

model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(l2_rate), input_shape=input_shape))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(l2_rate)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))


model.add(Conv2D(256, (3, 3), activation='relu', kernel_regularizer=l2(l2_rate)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

batch_size = 16
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    './Dataset4/train',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='binary',
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_directory(
    './Dataset4/test',
    target_size = input_shape[:2],
    batch_size = batch_size,
    class_mode='binary',
    color_mode='grayscale'
)

results = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
    callbacks = [lr_schedule, early_stoping, learning_rate_reduction]
)

model_save_path = './alkonost-eye.keras'
model.save(model_save_path)

Found 4020 images belonging to 2 classes.
Found 644 images belonging to 2 classes.
Epoch 1/20
252/252 [==============================] - 943s 4s/step - loss: 2.4347 - accuracy: 0.5903 - val_loss: 1.7685 - val_accuracy: 0.6832 - lr: 0.0010
Epoch 2/20
252/252 [==============================] - 939s 4s/step - loss: 1.2124 - accuracy: 0.6224 - val_loss: 1.1520 - val_accuracy: 0.5419 - lr: 0.0010
Epoch 3/20
252/252 [==============================] - 940s 4s/step - loss: 0.8994 - accuracy: 0.6557 - val_loss: 0.6700 - val_accuracy: 0.7795 - lr: 0.0010
Epoch 4/20
252/252 [==============================] - 939s 4s/step - loss: 0.7198 - accuracy: 0.6811 - val_loss: 0.8061 - val_accuracy: 0.5404 - lr: 0.0010
Epoch 5/20
252/252 [==============================] - 951s 4s/step - loss: 0.6492 - accuracy: 0.6950 - val_loss: 0.7203 - val_accuracy: 0.6289 - lr: 0.0010
Epoch 6/20
252/252 [==============================] - 939s 4s/step - loss: 0.6347 - accuracy: 0.7067 - val_loss: 3.7346 - val_accuracy: 0

In [2]:
import matplotlib.pyplot as plt
from keras.models import load_model
import cv2

model_save_path = './alkonost-eye.keras'

loaded_model = load_model(model_save_path)
prediction_results = []
input_shape=(320, 320, 1)
reshaped_shape = (1, 320, 320, 1)
for i in range(0, 11):
    #orks_img = cv2.imread(f'./manual-check-data/no-orks/{i}.png', 0) if not cv2.imread(f'./manual-check-data/no-orks/{i}.png', 0) is None else cv2.imread(f'./manual-check-data/no-orks/{i}.jpg', 0)
    orks_img = cv2.imread(f'./manual-check-data/orks/{i}.png', 0) if not cv2.imread(f'./manual-check-data/orks/{i}.png', 0) is None else cv2.imread(f'./manual-check-data/orks/{i}.jpg', 0)
    orks_img = cv2.resize(orks_img, input_shape[:2]).reshape(reshaped_shape)
    print(loaded_model.predict(orks_img))
    prediction_results.append(f'{i} - {loaded_model.predict(orks_img)}')
print(prediction_results)

1/1 [==============================] - 1s 571ms/step
[[1.]]
1/1 [==============================] - 0s 129ms/step
[[1.]]
1/1 [==============================] - 0s 135ms/step
[[1.]]
1/1 [==============================] - 0s 135ms/step
[[0.]]
1/1 [==============================] - 0s 135ms/step
[[0.00011518]]
1/1 [==============================] - 0s 135ms/step
[[1.00429206e-17]]
1/1 [==============================] - 0s 141ms/step
[[1.]]
1/1 [==============================] - 0s 134ms/step
[[1.]]
1/1 [==============================] - 0s 133ms/step
[[1.]]
1/1 [==============================] - 0s 136ms/step
[[1.]]
1/1 [==============================] - 0s 134ms/step
[[1.]]
1/1 [==============================] - 0s 139ms/step
['0 - [[1.]]', '1 - [[1.]]', '2 - [[1.]]', '3 - [[0.]]', '4 - [[0.00011518]]', '5 - [[1.00429206e-17]]', '6 - [[1.]]', '7 - [[1.]]', '8 - [[1.]]', '9 - [[1.]]', '10 - [[1.]]']
